In [1]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 16.4 MB/s eta 0:00:00


In [2]:
from diffusers import UNet2DConditionModel
import torch

class ModifiedUNet(UNet2DConditionModel):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # unet = super().from_pretrained("segmind/tiny-sd", subfolder="unet")
        self.register_to_config(cross_attention_dim=256)

        for down_block in self.down_blocks:
            for attn in down_block.attentions:
                for transformer_block in attn.transformer_blocks:
                    if hasattr(transformer_block, "attn2"):
                        transformer_block.attn2.to_k = torch.nn.Linear(256, transformer_block.attn2.to_k.out_features, bias=False)
                        transformer_block.attn2.to_v = torch.nn.Linear(256, transformer_block.attn2.to_v.out_features, bias=False)

        for up_block in self.up_blocks:
            for attn in up_block.attentions:
                for transformer_block in attn.transformer_blocks:
                    if hasattr(transformer_block, "attn2"):
                        transformer_block.attn2.to_k = torch.nn.Linear(256, transformer_block.attn2.to_k.out_features, bias=False)
                        transformer_block.attn2.to_v = torch.nn.Linear(256, transformer_block.attn2.to_v.out_features, bias=False)

        if not ("target_modules" in kwargs):
            target_modules = ["to_k", "to_q", "to_v", "to_out.0"]

        rank = kwargs["rank"] if "rank" in kwargs else 64
        if "use_lora" in kwargs and kwargs["use_lora"]:
            self.lora_config = LoraConfig(
                r=rank,
                use_dora=(kwargs["use_dora"] if "use_dora" in kwargs else False),
                lora_alpha=rank,
                init_lora_weights="gaussian",
                lora_dropout=(kwargs["lora_dropout"] if "lora_dropout" in kwargs else None),
                target_modules=self.target_modules,
                use_rslora=(kwargs["use_rslora"] if "use_rslora" in kwargs else None),
            )

            self.add_adapter(self.lora_config)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
selfx = UNet2DConditionModel.from_pretrained("segmind/tiny-sd", subfolder="unet", in_channels=260, low_cpu_mem_usage=False, ignore_mismatched_sizes=True)

selfx.register_to_config(cross_attention_dim=256)

for down_block in selfx.down_blocks:
    for attn in down_block.attentions:
        for transformer_block in attn.transformer_blocks:
            if hasattr(transformer_block, "attn2"):
                transformer_block.attn2.to_k = torch.nn.Linear(256, transformer_block.attn2.to_k.out_features, bias=False)
                transformer_block.attn2.to_v = torch.nn.Linear(256, transformer_block.attn2.to_v.out_features, bias=False)

for up_block in selfx.up_blocks:
    for attn in up_block.attentions:
        for transformer_block in attn.transformer_blocks:
            if hasattr(transformer_block, "attn2"):
                transformer_block.attn2.to_k = torch.nn.Linear(256, transformer_block.attn2.to_k.out_features, bias=False)
                transformer_block.attn2.to_v = torch.nn.Linear(256, transformer_block.attn2.to_v.out_features, bias=False)

selfx.save_pretrained("tiny-sd-local", safe_serialization=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


unet/config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

An error occurred while trying to fetch segmind/tiny-sd: segmind/tiny-sd does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


diffusion_pytorch_model.bin:   0%|          | 0.00/647M [00:00<?, ?B/s]

In [7]:
selfx

UNet2DConditionModel(
  (conv_in): Conv2d(260, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0): Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in_features=320, out_featur

In [ ]:
config = selfx.config
unet = UNet2DConditionModel(**config)
import torch
unet.load_state_dict(torch.load("/content/tiny-sd-local/diffusion_pytorch_model.bin"))

<ipython-input-4-354d7e7d424f>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet.load_state_dict(torch.load("/content/tiny-sd-local/diffusion_pytorch_model.bin"))


<All keys matched successfully>

In [ ]:
input_tensor = torch.randn(1, 4, 52, 45)
unet

with torch.no_grad():
  x = unet(input_tensor, timestep=1000, encoder_hidden_states=torch.randn(1, 1, 256))

In [ ]:
x.sample.shape